In [1]:
import time
from dataclasses import dataclass, field

# from pathlib import Path
from typing import ClassVar

# import pandas as pd
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [5]:
@dataclass(frozen=True)
class Gamer:
    """Gamer."""

    name: str
    gamer_tag: str
    url: str = field(init=False)
    base_url: ClassVar[str] = "https://mlb22.theshow.com/universal_profiles"
    endpoint: ClassVar[str] = "game_history"
    query: ClassVar[str] = "?mode=ARENA"

    def __post_init__(self) -> None:
        """Build the url."""
        self.url = f"{self.base_url}/{self.gamer_tag}/{self.endpoint}{self.query}"

In [3]:
gamers = {
    "chris": "cdp0089",
    "kazem": "UOKazem",
    "jalil": "jalidoost2424",
}
chris, kazem, jalil = tuple(
    [Gamer(name, gamer_tag) for name, gamer_tag in gamers.items()]
)

In [4]:
print(chris)
print(kazem)
print(jalil)

Gamer(name='chris', gamer_tag='cdp0089', url='https://mlb22.theshow.com/universal_profiles/cdp0089/game_history?mode=ARENA')
Gamer(name='kazem', gamer_tag='UOKazem', url='https://mlb22.theshow.com/universal_profiles/UOKazem/game_history?mode=ARENA')
Gamer(name='jalil', gamer_tag='jalidoost2424', url='https://mlb22.theshow.com/universal_profiles/jalidoost2424/game_history?mode=ARENA')


In [3]:
# Set up Splinter (prep the automated browser).
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)

In [4]:
# Use in time.sleep() to allow for pages and tables to load
delay_time = 4

In [5]:
browser.visit(chris.url)
# Delay to allow the page to load.
time.sleep(delay_time)

# Retrieve html.
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [12]:
games_meta_data = soup.find_all("div", attrs={"class": "mlb22-games-box"})

p_tags = games_meta_data[0].find_all("p")
game_date = p_tags[1].text

table_rows = games_meta_data[0].find_all("tr")
table_rows.pop(0)

game_teams = [row.find_all("td")[1].text for row in table_rows]
game_gamer_tags = [row.find_all("td")[2].text.strip() for row in table_rows]
if "CPU" in game_teams:
    print("skip this box score")

# if (game_gamer_tags[0] in chris.gamer_tag.values()) and (
#     game_gamer_tags[1] in gamer_tags.values()
# ):
#     print("this is a Head-To-Head game")